**Installation and imports**

In [ ]:
!pip install openai langchain==0.1.14
!pip install psycopg2

In [ ]:
import os
import psycopg2
from psycopg2.extras import DictCursor
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.evaluation import load_evaluator

**Define inference function**

In [ ]:
def perform_inference_with_template(input_text, template):
    llm = Ollama(
        base_url="http://llm.ic-shared-llm.svc.cluster.local:11434",
        model="mistral",
        top_p=0.95,
        temperature=0.01,
        num_predict=512,
        repeat_penalty=1.03,
        callbacks=[StreamingStdOutCallbackHandler()]
    )

    PROMPT = PromptTemplate.from_template(template)
    llm_chain = LLMChain(llm=llm, prompt=PROMPT)
    return llm_chain.run(input_text)

**Get PostgreSQL database connection parameters from environment variables**

In [ ]:
conn_params = {
    "dbname": os.getenv('DATABASE_NAME'),
    "user": os.getenv('DATABASE_USER'),
    "password": os.getenv('DATABASE_PASS'),
    "host": os.getenv('DATABASE_HOST'),
    "port": os.getenv('DATABASE_PORT')
}

**Define queries**

In [ ]:
incident_id = os.getenv('INCIDENT_ID')
entity_name = os.getenv('ENTITY_TO_PROCES')

In [ ]:
select_record_by_id_query = """
SELECT * 
FROM fracture_incidents
WHERE id = %s;
"""

update_record_query = f"""
UPDATE fracture_incidents
SET 
    {entity_name} = %s
WHERE 
    id = %s;
"""

**Query database to get incident data**

In [ ]:
record = {}

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor(cursor_factory=DictCursor)

    cursor.execute(select_record_by_id_query, (incident_id,))
    record = cursor.fetchone()
    
    if record:
        print("Retrieved record:")
        print(record)
    else:
        print(f"No record found with ID {incident_id}.")

except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
finally:
    if conn:
        cursor.close()
        conn.close()
        print("Database connection closed.")

**Read the template for this node**

In [ ]:
with open(f"templates/{entity_name}_template.txt") as f:
        template = f.read()

**Use the template and the data from database with the LLM**

In [ ]:
entity = perform_inference_with_template(record['description'], template)

**Update database with result from the LLM**

In [ ]:
update_data = (
    entity,
    incident_id
)

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    cursor.execute(update_record_query, update_data)

    conn.commit()
    print(f"Record with ID {incident_id} updated successfully.")

except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    if conn:
        cursor.close()
        conn.close()
        print("Database connection closed.")